In [1]:
import pandas as pd
import numpy as np
import tsfresh
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import tensorflow as tf

import os

import matplotlib.pyplot as plt

# keras goodies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, Dropout, MaxPooling1D, BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import metrics as kmetrics
import tensorflow.keras.backend as K

C:\Users\78182\Anaconda3\envs\pdiot\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\78182\Anaconda3\envs\pdiot\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\78182\Anaconda3\envs\pdiot\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import os
clean_data_folder = "./Data/Clean"
for dirpath, dirnames, filenames in os.walk(clean_data_folder):
    print(dirpath)
    print(filenames)

./Data/Clean
['Respeck_s1711661_Climbing stairs_23-09-2021_20-59-28.csv', 'Respeck_s1711661_Descending stairs_23-09-2021_20-58-29.csv', 'Respeck_s1711661_Desk work_23-09-2021_21-09-26.csv', 'Respeck_s1711661_Falling on knees_24-09-2021_21-30-28.csv', 'Respeck_s1711661_Falling on the back_24-09-2021_21-50-51.csv', 'Respeck_s1711661_Falling on the left_24-09-2021_21-38-30.csv', 'Respeck_s1711661_Falling on the right_24-09-2021_21-42-19.csv', 'Respeck_s1711661_Lying down left_23-09-2021_20-10-11.csv', 'Respeck_s1711661_Lying down on back_23-09-2021_20-06-21.csv', 'Respeck_s1711661_Lying down on stomach_23-09-2021_20-15-13.csv', 'Respeck_s1711661_Lying down right_23-09-2021_20-12-34.csv', 'Respeck_s1711661_Movement_23-09-2021_21-12-32.csv', 'Respeck_s1711661_Running_23-09-2021_20-44-06.csv', 'Respeck_s1711661_Sitting bent backward_23-09-2021_19-56-37.csv', 'Respeck_s1711661_Sitting bent forward_23-09-2021_19-54-06.csv', 'Respeck_s1711661_Sitting_23-09-2021_19-51-48.csv', 'Respeck_s1711661_

In [3]:
base_df = pd.DataFrame()

clean_data_folder = "./Data/Clean"

for dirpath, dirnames, filenames in os.walk(clean_data_folder):
    if len(filenames) != 0:
        for f in filenames:
            full_path = f"{dirpath}/{f}"
            #Only load data from respeck
            if "Respeck" in full_path: 
                # load data into a DataFrame
                new_df = pd.read_csv(full_path)
                # merge into the base DataFrame
                base_df = pd.concat([base_df, new_df])

In [4]:
len(base_df)

756203

In [5]:
base_df.reset_index(drop=True, inplace=True)
base_df

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,sensor_type,activity_type,activity_code,subject_id,notes,recording_id
0,1.632427e+12,-0.001221,-1.730286,0.314636,-5.671875,-0.781250,-5.390625,Respeck,Climbing stairs,12,s1711661,NaN,Respeck_s1711661_Climbing stairs_23-09-2021_20...
1,1.632427e+12,0.402588,-0.721497,-0.363586,-11.187500,8.390625,2.437500,Respeck,Climbing stairs,12,s1711661,NaN,Respeck_s1711661_Climbing stairs_23-09-2021_20...
2,1.632427e+12,0.178223,-0.786194,-0.272766,20.531250,-0.546875,7.984375,Respeck,Climbing stairs,12,s1711661,NaN,Respeck_s1711661_Climbing stairs_23-09-2021_20...
3,1.632427e+12,0.012939,-1.108215,-0.015686,22.875000,13.265625,1.203125,Respeck,Climbing stairs,12,s1711661,NaN,Respeck_s1711661_Climbing stairs_23-09-2021_20...
4,1.632427e+12,-0.034912,-0.832825,0.010681,16.484375,11.187500,1.296875,Respeck,Climbing stairs,12,s1711661,NaN,Respeck_s1711661_Climbing stairs_23-09-2021_20...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
756198,1.632940e+12,-0.304199,-1.083313,0.414978,-0.578125,18.859375,-18.781250,Respeck,Walking at normal speed,1,s2254050,NaN,Respeck_s2254050_Walking at normal speed_29-09...
756199,1.632940e+12,-0.057129,-1.235901,0.121765,1.593750,14.609375,-2.328125,Respeck,Walking at normal speed,1,s2254050,NaN,Respeck_s2254050_Walking at normal speed_29-09...
756200,1.632940e+12,0.239258,-1.449768,-0.063782,-6.281250,22.671875,18.984375,Respeck,Walking at normal speed,1,s2254050,NaN,Respeck_s2254050_Walking at normal speed_29-09...
756201,1.632940e+12,0.305908,-1.131653,-0.114075,-3.718750,11.906250,13.140625,Respeck,Walking at normal speed,1,s2254050,NaN,Respeck_s2254050_Walking at normal speed_29-09...


## Formatting recording_id (Assign a groupId based on subject_id and activity_type)
There are some ill formated recording_id egg, same activity should have same recording_id, the following line will create unique id out of (subject_id,activity_type), thereby fixing the ill formated recording_id

In [6]:
base_df['grpId']=base_df.groupby(['activity_type','subject_id']).ngroup().add(1)

## create window id for each activity/student_id
each window_id is a training sample for a specific activity.

each window_id represent a training input for an activity, one activity can have multiple window_id.

In [7]:
window_size = 50 # 50 datapoints for the window size, which, at 25Hz, means 2 seconds
step_size = 25 # this is 50% overlap

window_number = 0 # start a counter at 0 to keep track of the window number

all_overlapping_windows = []

for gid, group in base_df.groupby("grpId"):
    
    large_enough_windows = [window for window in group.rolling(window=window_size, min_periods=window_size) if len(window) == window_size]
    
    overlapping_windows = large_enough_windows[::step_size] 
    
    # then we will append a window ID to each window
    for window in overlapping_windows:
        window.loc[:, 'window_id'] = window_number
        window_number += 1


    all_overlapping_windows.append(pd.concat(overlapping_windows).reset_index(drop=True))

In [8]:
final_sliding_windows = pd.concat(all_overlapping_windows).reset_index(drop=True)

Taking a look at activity for each activity

In [9]:
final_sliding_windows.groupby("window_id")[['activity_type']].agg(np.min)

,activity_type
window_id,
0,Climbing stairs
1,Climbing stairs
2,Climbing stairs
3,Climbing stairs
4,Climbing stairs
...,...
29037,Walking at normal speed
29038,Walking at normal speed
29039,Walking at normal speed


In [10]:
final_sliding_windows.groupby("window_id")[['activity_type']].agg(np.min)['activity_type'].unique()

array(['Climbing stairs', 'Descending stairs', 'Desk work',
       'Falling on knees', 'Falling on the back', 'Falling on the left',
       'Falling on the right', 'Lying down left', 'Lying down on back',
       'Lying down on stomach', 'Lying down right', 'Movement', 'Running',
       'Sitting', 'Sitting bent backward', 'Sitting bent forward',
       'Standing', 'Walking at normal speed'], dtype=object)

Assign class label for every activity, later we will convert them to one-hot encoding

In [11]:
class_labels = {
    'Desk work': 0,
    'Walking at normal speed': 1,
    'Climbing stairs': 2,
    'Descending stairs':3,
    'Falling on knees': 4,
    'Falling on the back': 5,
    'Falling on the left': 6,
    'Falling on the right': 7,
    'Lying down left': 8,
    'Lying down on back': 9,
    'Lying down on stomach': 10,
    'Lying down right': 11,
    'Movement': 12,
    'Running': 13,
        'Sitting': 14,
        'Sitting bent backward': 15,
        'Sitting bent forward': 16,
    'Standing': 17
}

In [12]:
window_id_class_labels = final_sliding_windows.groupby("window_id")[['activity_type']].agg(np.min).replace(class_labels)
window_id_class_labels

,activity_type
window_id,
0,2
1,2
2,2
3,2
4,2
...,...
29037,1
29038,1
29039,1


In [13]:
window_id_class_labels.groupby("window_id")[['activity_type']].agg(np.min)['activity_type'].unique()

array([ 2,  3,  0,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        1], dtype=int64)

Lastly, we are only interested in the following columns

In [14]:
columns_of_interest = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']

## Training a simple CNN

In [15]:
filters = 64
kernel_size = 20
n_features = 6
activation='relu'
n_classes = 18

model = Sequential()

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear', 
                 input_shape=(window_size, n_features)))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='linear'))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 31, 64)            7744      
_________________________________________________________________
batch_normalization (BatchNo (None, 31, 64)            256       
_________________________________________________________________
activation (Activation)      (None, 31, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 12, 64)            81984     
_________________________________________________________________
batch_normalization_1 (Batch (None, 12, 64)            256       
_________________________________________________________________
activation_1 (Activation)    (None, 12, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 768)               0

In [16]:
X = []
y = []

for window_id, group in final_sliding_windows.groupby('window_id'):
    
    shape = group[columns_of_interest].values.shape
    
    X.append(group[columns_of_interest].values)
    y.append(class_labels[group["activity_type"].values[0]])

In [31]:
X = np.asarray(X)
y = np.asarray(y)
X = tf.keras.utils.normalize(X, axis=1)
print(f"X shape = {X.shape}")
print(f"y shape = {y.shape}")

X shape = (29042, 50, 6)
y shape = (29042,)


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, train_size=0.8)


In [154]:
model.compile(
    optimizer=optimizers.SGD(lr=0.0005),
    loss='categorical_crossentropy',
    metrics = ['accuracy'])

In [156]:
model.fit(X_train, y_train,
        batch_size=20, epochs=100)

Epoch 1/100
428/987 [============>.................] - ETA: 2s - loss: nan - accuracy: 0.0548

KeyboardInterrupt: 

In [132]:
# stats
y_pred_ohe = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

In [133]:

print("*" * 80)
print("Classification report")
print("*" * 80)
print(classification_report(y_true_labels, y_pred_labels))

********************************************************************************
Classification report
********************************************************************************
              precision    recall  f1-score   support

           0       0.85      0.24      0.37       314
           1       0.66      0.83      0.74       259
           2       0.66      0.64      0.65       264
           3       0.64      0.63      0.64       227
           4       0.79      0.78      0.79       303
           5       0.81      0.79      0.80       272
           6       0.94      0.77      0.85       247
           7       0.83      0.84      0.84       301
           8       0.83      0.88      0.85       289
           9       0.82      0.85      0.83       288
          10       0.82      0.86      0.84       265
          11       0.85      0.90      0.88       301
          12       0.63      0.59      0.61       246
          13       0.96      0.94      0.95       290
     

## Trying Auto-CNN

In [34]:
import os
import tensorflow as tf
from auto_cnn.gan import AutoCNN
import random
random.seed(42)
tf.random.set_seed(42)
data = {'x_train': X_train, 'y_train': y_train, 'x_test': X_test, 'y_test': y_test}
a = AutoCNN(population_size=5, maximal_generation_number=4, dataset=data, epoch_number=5)
best_cnn = a.run()
print(best_cnn)

Initializing Population
Population Initialization Done: [128-64, 32-512-256-32, 64-512, 64-512-256-512-32-64-max-128-32, 128-128]
Generation 0
Evaluating Population fitness
128-64 0.0580134280025959
[32-512, 256-32]
Epoch 1/5
291/291 [==============================] - 476s 2s/step - loss: 1.5159 - accuracy: 0.4917 - val_loss: nan - val_accuracy: 0.0501
Epoch 2/5
291/291 [==============================] - 445s 2s/step - loss: 1.0388 - accuracy: 0.6250 - val_loss: nan - val_accuracy: 0.2763
Epoch 3/5
291/291 [==============================] - 394s 1s/step - loss: 0.8660 - accuracy: 0.6845 - val_loss: nan - val_accuracy: 0.6083
Epoch 4/5
291/291 [==============================] - 425s 1s/step - loss: 0.7544 - accuracy: 0.7255 - val_loss: nan - val_accuracy: 0.5636
Epoch 5/5
91/91 [==============================] - 24s 265ms/step - loss: nan - accuracy: 0.6309
32-512-256-32 0.6309175491333008
[64-512]
Epoch 1/5
291/291 [==============================] - 136s 467ms/step - loss: 24.3350 - ac

Epoch 1/5
291/291 [==============================] - 85s 290ms/step - loss: 23.7994 - accuracy: 0.4426 - val_loss: nan - val_accuracy: 0.0581
Epoch 2/5
291/291 [==============================] - 81s 279ms/step - loss: 0.9390 - accuracy: 0.6872 - val_loss: nan - val_accuracy: 0.2655
Epoch 3/5
291/291 [==============================] - 82s 281ms/step - loss: 0.6437 - accuracy: 0.7768 - val_loss: nan - val_accuracy: 0.5821
Epoch 4/5
291/291 [==============================] - 81s 279ms/step - loss: 0.4711 - accuracy: 0.8411 - val_loss: nan - val_accuracy: 0.6525
Epoch 5/5
91/91 [==============================] - 7s 75ms/step - loss: nan - accuracy: 0.6497
32-512 0.6496815085411072
Evaluating Offsprings Done: {'128-64': 0.0580134280025959, '32-512-256-32': 0.6309175491333008, '64-512': 0.6622482538223267, '64-512-256-512-32-64-max-128-32': 0.6951282620429993, '128-128': 0.5023239850997925, '64-512-64-512-256-512-32-64-max-128-32': 0.05284902825951576, '': 0.3339645266532898, '128-32': 0.650

291/291 [==============================] - 626s 2s/step - loss: 2.8905 - accuracy: 0.0555 - val_loss: 2.8977 - val_accuracy: 0.0544
Epoch 4/5
291/291 [==============================] - 649s 2s/step - loss: 2.8905 - accuracy: 0.0561 - val_loss: 2.8978 - val_accuracy: 0.0544
Epoch 5/5
91/91 [==============================] - 37s 406ms/step - loss: 2.8962 - accuracy: 0.0528
64-512-256-512-max-128-32 0.05284902825951576
[128-128, 32-64, max, 128-32]
Epoch 1/5
291/291 [==============================] - 95s 323ms/step - loss: 1.4800 - accuracy: 0.5123 - val_loss: 27.3161 - val_accuracy: 0.0596
Epoch 2/5
291/291 [==============================] - 93s 321ms/step - loss: 0.9648 - accuracy: 0.6517 - val_loss: 13.2767 - val_accuracy: 0.1564
Epoch 3/5
291/291 [==============================] - 94s 325ms/step - loss: 0.8215 - accuracy: 0.7006 - val_loss: 0.8322 - val_accuracy: 0.6966
Epoch 4/5
291/291 [==============================] - 97s 334ms/step - loss: 0.6827 - accuracy: 0.7485 - val_loss: 0.

291/291 [==============================] - 57s 195ms/step - loss: 0.6775 - accuracy: 0.7528 - val_loss: 0.8629 - val_accuracy: 0.6934
Epoch 5/5
91/91 [==============================] - 4s 41ms/step - loss: 0.7223 - accuracy: 0.7488
128-128-max-32-32 0.7488380074501038
Evaluating Offsprings Done: {'128-64': 0.0580134280025959, '32-512-256-32': 0.6309175491333008, '64-512': 0.6622482538223267, '64-512-256-512-32-64-max-128-32': 0.6951282620429993, '128-128': 0.5023239850997925, '64-512-64-512-256-512-32-64-max-128-32': 0.05284902825951576, '': 0.3339645266532898, '128-32': 0.6500258445739746, '32-512-256-32-64-512': 0.5248751640319824, '256-128-64-512-256-32': 0.5611981153488159, '32-512': 0.6496815085411072, 'max-64-512-128-128': 0.4759855270385742, '256-512-32-64-max-256-512-128-32': 0.6897916793823242, '64-512-256-512-32-64': 0.05284902825951576, '128-128-max-128-32': 0.7319676280021667, '64-512-256-512-max-128-32': 0.05284902825951576, '128-128-32-64-max-128-32': 0.726286768913269, '

5809

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 